In [3]:
import requests
from bs4 import BeautifulSoup
import json
import pandas as pd
import gspread
from oauth2client.service_account import ServiceAccountCredentials
from selenium import webdriver

def return_club_info(url):
    page = requests.get(url)
    
    soup = BeautifulSoup(page.content, "html5lib")
    content = soup.find('div', {'class':"engage-application"}).find('script')
    json_raw = content.text
    json_raw = json_raw[25:-1]
    
    club_info = json.loads(json_raw)
    
    fields = [club_info['preFetchedData']['organization']['name'],
          club_info['preFetchedData']['organization']['email'],
          club_info['preFetchedData']['organization']['primaryContact']['firstName'],
          club_info['preFetchedData']['organization']['primaryContact']['lastName'],
          club_info['preFetchedData']['organization']['primaryContact']['primaryEmailAddress'],
          club_info['preFetchedData']['organization']['contactInfo'][0]['phoneNumber'],
          url,
          club_info['preFetchedData']['organization']['socialMedia']['externalWebsite'],
          club_info['preFetchedData']['organization']['socialMedia']['tumblrUrl'],
          club_info['preFetchedData']['organization']['socialMedia']['facebookUrl'],
          club_info['preFetchedData']['organization']['socialMedia']['instagramUrl'],
          club_info['preFetchedData']['organization']['socialMedia']['twitterUrl'],
          club_info['preFetchedData']['organization']['socialMedia']['twitterUserName']]
    
    for index in range(len(fields)):
        if fields[index] == None:
            fields[index] = ''
    
    contact_us_form_url = ''

    if url[-1] != '/':
        contact_us_form_url = url + '/contact'
    else:
        contact_us_form_url = url + 'contact'

    data = {
        'State':'',
        'Volunteer':'',
        'University':'',
        'Department/Club Name':[fields[0]],
        'Department/Club Email':[fields[1]],
        'Contact Name':[fields[2] + ' ' + fields[3]],
        'Contact Email':[fields[4]],
        'Contact Phone Number':[fields[5]],
        'Comments':[contact_us_form_url]
    }

    df = pd.DataFrame(data)
    if fields[-1] != '' and fields[-1] != None:
        fields[-1] = 'https://twitter.com/' + fields[-1]

    for index in range(-6, 0):
        if fields[index] == '' or fields[index] == None:
            fields.pop(index)
    
    for social_media_contact in fields[6:]:
        df_row = df.iloc[0,:].copy()
        df_row['Comments'] = social_media_contact
        df = df.append([df_row], ignore_index=True)
    
    scope = ['https://spreadsheets.google.com/feeds',
             'https://www.googleapis.com/auth/drive']
    credentials = ServiceAccountCredentials.from_json_keyfile_name(
        'service_account.json', scope)

    spreadsheetId = '1qsAF7EmTnFiiDc9Wamkm54dAy4SAjIVvk2DzR7JQbA4'  
    sheetName = 'Master' 

    gc = gspread.authorize(credentials)
    sh = gc.open_by_key(spreadsheetId)
    
    values = df.values.tolist()
    sh.values_append(sheetName, {'valueInputOption': 'USER_ENTERED'}, {'values': values})
    
    return df

In [4]:
browser = webdriver.Chrome()

browser.get("https://utrgv.campuslabs.com/engage/organization/continuing")

return_club_info('https://utrgv.campuslabs.com/engage/organization/continuing')

TypeError: 'NoneType' object is not subscriptable

In [ ]:
#root_url = "https://uh.campuslabs.com/engage/organizations/"
root_url = "https://utrgv.campuslabs.com/engage/organizations/"

def add_query_to_root_url(query="health"):
    root_url_query = root_url + "?query=" + query
    return root_url_query

query_list = ["health", "mental", "medical"]

for query in query_list:
    browser = webdriver.Chrome()

    browser.get(add_query_to_root_url(query))

    elems = browser.find_elements_by_css_selector("ul a[href]")
    links = [elem.get_attribute('href') for elem in elems]
    
    for link in links:
        print(link)
        URL = link
        return_club_info(URL)

In [2]:
import types
def imports():
    for name, val in globals().items():
        if isinstance(val, types.ModuleType):
            yield val.__name__
list(imports())

['builtins',
 'builtins',
 'requests',
 'json',
 'pandas',
 'gspread',
 'selenium.webdriver',
 'types']

In [8]:
def return_club_info(url):
    page = requests.get(url)
    
    soup = BeautifulSoup(page.content, "html5lib")
    content = soup.find('div', {'class':"engage-application"}).find('script')
    json_raw = content.text
    json_raw = json_raw[25:-1]
    
    print(json.loads(json_raw))

return_club_info("https://utrgv.campuslabs.com/engage/organization/leadershape")

{'institution': None, 'coverPhoto': {'id': 111740, 'imageId': '92f06ae7-9052-4d6b-9708-c51b5d93d874', 'imagePath': '71a6ce37-adc1-4251-89b9-351873e2672546d7adc0-d281-4b5b-8c3d-c6fa717de562.png', 'url': 'https://se-images-blob.campuslabs.com/communities/4/coverphotos/92f06ae7-9052-4d6b-9708-c51b5d93d874.jpg', 'thumbnailUrl': 'https://se-images-blob.campuslabs.com/communities/4/coverphotos/92f06ae7-9052-4d6b-9708-c51b5d93d874-thumbnail.jpg', 'caption': 'Uploaded cover photo has no appropriate alternative text', 'dateCreated': '2018-01-18T12:09:42-05:00', 'institutionId': 4}, 'user': None, 'preFetchedData': {'event': None, 'loadAdditionalEventDetails': False, 'organization': {'id': 59995, 'communityId': 252, 'institutionId': 4, 'name': 'LeaderShape Institute ', 'shortName': 'LeaderShape', 'nameSortKey': 'L', 'websiteKey': 'leadershape', 'email': 'leadership@utrgv.edu', 'description': '<p><strong>2017 Application:</strong></p>\r\n<p><strong><a href="https://www.leadershape.org/Account/Logi